<a href="https://colab.research.google.com/github/cardstdani/sql-storage/blob/main/PostgreSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title **Click to install dependencies (PostgreSQL DBMS and *faker* library for Python)**
# @markdown If checked, installation output will be suppressed unless there are errors (warnings are always shown).
SILENT_INSTALL = True # @param {type:"boolean"}

redirect = "> /dev/null 2>&1" if SILENT_INSTALL else ""

!sudo apt-get install -y postgresql-common {redirect}
!sudo /usr/share/postgresql-common/pgdg/apt.postgresql.org.sh -y {redirect}
!apt-get update -qq && apt-get install -qq -y postgresql postgresql-contrib {redirect}
!pip install faker {redirect}

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
# @title **Start PostgreSQL service**
!service postgresql start

 * Starting PostgreSQL 17 database server
   ...done.


**Hello world in SQL**

To execute **SQL commands** on the DBMS, we can use the following **shell command**, where we write the SQL command we want to execute between the quotation marks "".

In [3]:
!sudo -u postgres psql -P pager=off -c "SELECT 'Hello World' AS HelloWorldColumn;"

 helloworldcolumn 
------------------
 Hello World
(1 row)



**Get DBMS and machine version**

The `-P pager=off` option is used with the `psql` command to disable the pager. By default, `psql` uses a pager (like `less` or `more`) to display results when the output is too large to fit on the screen. Setting `pager=off` ensures that the entire output of the command is printed directly to the standard output without being sent to a **pager**. This is useful in scripts or notebooks where you want to see the full output of a SQL command.

- https://stackoverflow.com/questions/14474138/customize-pager-in-psql

Then, to get the version of the DBMS being used, call the `version()` function:

In [4]:
!sudo -u postgres psql -P pager=off -c "SELECT version();"

                                                              version                                                              
-----------------------------------------------------------------------------------------------------------------------------------
 PostgreSQL 17.6 (Ubuntu 17.6-1.pgdg22.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit
(1 row)



**Drop database**

First, we will create a database called `ExampleDataBase`. Before doing anything else, we decide to use a command to delete the database named `ExampleDataBase`, because if it exists, we will erase all its content to create it from scratch. This operation is **critical**, as it deletes all the content of the previously created database named `ExampleDataBase`.

In [5]:
!sudo -u postgres psql -c "DROP DATABASE ExampleDataBase;"

ERROR:  database "exampledatabase" does not exist


**Database creation**

Now, we create the database called `ExampleDataBase` and declare that its **owner** user is `postgres`. This is done because in the shell commands we are executing, we are specifying to the DBMS that the user we are accessing the DBMS with is `postgres` using the `-u` option.

In [6]:
!sudo -u postgres psql -c "CREATE DATABASE ExampleDataBase OWNER postgres;"

CREATE DATABASE


**DDL script execution**

With the database created, we need to run the **DDL** code to initialize the tables, constraints, and other elements of our logical model. For this, in a local **PostgreSQL** installation, we would connect to it using the command `\c ExampleDataBase` from the DBMS, but here we do it directly using the option `-d ExampleDataBase` in the shell commands we execute, so that the SQL code we run affects the database we just created.

In this way, first, we use **wget** to download a `schema.sql` file from **GitHub** with all the **DDL code**, so we don't have to copy and paste it into the DBMS. Then, we execute it with the option `-f schema.sql` in the following shell command. In a local DBMS installation, we could use `\i schema.sql`, or directly copy and paste the code into the DBMS terminal, although the latter is not recommended.

In [7]:
!wget -qO schema.sql https://gist.githubusercontent.com/cardstdani/1247573e1ef2f6ea9ab99b82c5761ad6/raw/schema.sql

!sudo -u postgres psql -q -d exampledatabase -f schema.sql

psql:schema.sql:1: NOTICE:  table "sportsanction" does not exist, skipping
psql:schema.sql:2: NOTICE:  table "poolsanction" does not exist, skipping
psql:schema.sql:3: NOTICE:  table "drivingsanction" does not exist, skipping
psql:schema.sql:4: NOTICE:  table "busticket" does not exist, skipping
psql:schema.sql:5: NOTICE:  table "participation" does not exist, skipping
psql:schema.sql:6: NOTICE:  table "carregistration" does not exist, skipping
psql:schema.sql:7: NOTICE:  table "membership" does not exist, skipping
psql:schema.sql:8: NOTICE:  table "cruisebooking" does not exist, skipping
psql:schema.sql:9: NOTICE:  table "residence" does not exist, skipping
psql:schema.sql:10: NOTICE:  table "entry" does not exist, skipping
psql:schema.sql:11: NOTICE:  table "bustrip" does not exist, skipping
psql:schema.sql:12: NOTICE:  table "team" does not exist, skipping
psql:schema.sql:13: NOTICE:  table "citybus" does not exist, skipping
psql:schema.sql:14: NOTICE:  table "carownership" does not

**Data generation and loading**

Once the tables and elements of the logical model are loaded into the database, we can populate it with synthetic data, meaning randomly generated data. However, it is also possible to populate it with other **data sources**. If any of these sources have a different **schema** than the elements of this database, an integration process should be followed so that their information can be inserted correctly.

In this case, we use the script below to generate a series of **CSV** files, specifically one for each table in our model, with the data of the tuples to insert. Additionally, we also generate a `load_data.sql` file that contains SQL statements to insert the data from the CSVs into the corresponding tables. For its generation, the script has two parameters that can be adjusted depending on our needs:

- **SEED:** This is a seed that determines the behavior of the random generator, so that each execution of the script is reproducible. It must be a **positive integer**.

- **FACTOR:** By default, the script generates a fixed number of tuples for each table to ensure all database **integrity constraints**, meaning all foreign keys point to existing and valid tuples, etc. However, if we want to generate more data than the script provides by default, we can increase the FACTOR parameter. For example, with a value of **FACTOR=2**, we are generating **twice** the data that is generated by default. It should be noted that the minimum value we can assign it is 1, and in a Google Collab environment, it is not recommended to use a FACTOR higher than 1500 due to the lack of RAM🤯.

In [8]:
# @title **Data generation**
import os
import random
import csv
from faker import Faker
from datetime import date, datetime, timedelta

# --------------------------------------------------
# CONFIGURACIÓN
# --------------------------------------------------
SEED = 10  # @param {"type":"integer"}
FACTOR = 200  # @param {"type":"integer"}
OUTPUT_DIR = '/content/data'           # Carpeta donde saldrán los CSVs
SQL_SCRIPT  = '/content/load_data.sql' # Script con COPYs

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------------------------------------
# Función de sanitización
# --------------------------------------------------
def safe_str(s, max_len=32):
    if s is None:
        return ''
    clean = s.replace('\n', ' ').replace('\r', ' ').replace("'", "''").strip()
    return clean[:max_len]

# --------------------------------------------------
# 1) Recuentos base por tabla
# --------------------------------------------------
BASE_COUNTS = {
    'CruiseLine': 5,
    'Person': 50,
    'City': 10,
    'Vehicle': None,
    'CruiseShip': 5,
    'Bike': 10,
    'Car': 10,
    'DrivingLicenseRequest': 20,
    'CarRegistration': 10,
    'CarOwnership': 10,
    'Rental': 10,
    'ShipAssignment': 10,
    'Port': 10,
    'Voyage': 20,
    'CruiseBooking': 20,
    'Residence': 20,
    'Pool': 10,
    'CruisePool': 5,
    'CityPool': 5,
    'OlympicPool': 5,
    'Entry': 20,
    'SwimmingCompetition': 5,
    'Team': 10,
    'Participation': 20,
    'Sanction': 20,
    'DrivingSanction': 10,
    'SportSanction': 5,
    'PoolSanction': 5,
    'Membership': 10,
    'CityBus': 10,
    'BusTrip': 20,
    'BusPass': 10,
    'BusTicket': 20
}

# --------------------------------------------------
# 2) Inicializar PRNG y Faker
# --------------------------------------------------
random.seed(SEED)
faker = Faker()
Faker.seed(SEED)

# --------------------------------------------------
# 3) Calcular recuentos reales
# --------------------------------------------------
counts = {}
for tbl, base in BASE_COUNTS.items():
    if base is not None:
        counts[tbl] = int(base * FACTOR)
# Vehicle es suma de las subclases
counts['Vehicle'] = (
    counts['CruiseShip']
    + counts['Bike']
    + counts['Car']
    + counts['CityBus']
)

# --------------------------------------------------
# 4) Definir columnas y abrir CSVs
# --------------------------------------------------
tables_columns = {
    'CruiseLine': ['Name','FoundationDate','ContactPhone','Rating'],
    'Person': ['Name','Birth','Email','Phone','Nationality','NameFK','FoundationDateFK'],
    'City': ['Name','Country','Population','Area','Latitude','Longitude'],
    'Vehicle': ['Model','Weight','Color','Odometer'],
    'CruiseShip': ['Speed','Length','PassengerCapacity','Class','VehicleID'],
    'Bike': ['Electric','Foldable','HasLights','FrameHeight','VehicleID'],
    'Car': ['Plate','FuelType','DoorCount','TrunkCapacity','HorsePower','Doors','AirConditioning','VehicleID'],
    'DrivingLicenseRequest': ['RequestDate','Fee','PersonFK'],
    'DrivingLicense': ['LicenseID','ApprovalDate','Points'],
    'RejectedDrivingLicense': ['LicenseID','RejectionDate','ReapplicationDate','Reason'],
    'CarRegistration': ['RegistrationDate','ExpirationDate','PlateFK','LicenseFK'],
    'CarOwnership': ['BuyDate','BuyPrice','WarrantyEndDate','OwnershipType','PlateFK','PersonFK'],
    'Rental': ['StartTimestamp','Duration','HourPrice','BikeFK','PersonFK'],
    'ShipAssignment': ['StartDate','EndDate','NameFK','FoundationDateFK','ShipFK'],
    'Port': ['Name','TerminalCount','MaxShipLength','Area','CityFK'],
    'Voyage': ['DepartureDate','ArrivalDate','Distance','DepartureNameFK','DepartureCityFK','ArrivalNameFK','ArrivalCityFK','ShipFK'],
    'CruiseBooking': ['BookingDate','CabinNumber','Price','PaymentMethod','PersonFK',
                      'DepartureDateFK','ArrivalDateFK','DepartureNameFK','DepartureCityFK','ArrivalNameFK','ArrivalCityFK','ShipFK'],
    'Residence': ['StartDate','EndDate','Address','PersonFK','CityFK'],
    'Pool': ['PoolID','Name','Address','MinDepth','MaxDepth','Status','CityFK'],
    'CruisePool': ['PoolID','DeckNumber','MaxCapacity','WaterTemperature','SlideCount','ShipFK'],
    'CityPool': ['PoolID','MaxCapacity','AnnualBudget','AccessibilityFeatures','FreeWifi'],
    'OlympicPool': ['PoolID','SpectatorMaxCapacity','CompetitionLanes'],
    'Entry': ['EntryTimestamp','Price','PaymentMethod','AppliedDiscount','Duration','PersonFK','PoolFK'],
    'SwimmingCompetition': ['Name','StartDate','EndDate','RecordTime','PrizeAmount'],
    'Team': ['Name','CreationDate','ClothColor','Sport','Budget','ContactEmail','CoachFK','HomePoolFK'],
    'Participation': ['RegistrationDate','Rank','RecordedTime','NameFK','StartDateFK','EndDateFK','TeamNameFK','CoachFK'],
    'Sanction': ['SanctionID','Amount','Description','IssueDate','ExpirationDate','Status'],
    'DrivingSanction': ['SanctionID','RecordedSpeed','PointsDeducted','SuspensionDays','PermanentSuspension','LicenseFK'],
    'SportSanction': ['SanctionID','SuspendedCompetitions','RefereeName','NameFK','StartDateFK','EndDateFK','TeamNameFK','CoachFK'],
    'PoolSanction': ['SanctionID','BanStartDate','BanEndDate','CompensationRequired','CommunityServiceHours','EntryFK','PersonFK','PoolFK'],
    'Membership': ['JoinDate','LeaveDate','FeeAmount','PaymentFrequency','AutoRenewal','PersonFK','TeamNameFK','CoachFK'],
    'CityBus': ['Plate','RouteNumber','Seats','FreeWifi','VehicleID','DriverFK','CityFK'],
    'BusTrip': ['TripDate','StartAddress','EndAddress','Duration','PlateFK'],
    'BusPass': ['PassID','IssueDate','ExpirationDate','Modality','RemainingTrips','PersonFK'],
    'BusTicket': ['IssueTime','TripDateFK','StartAddressFK','EndAddressFK','PlateFK','PersonFK','PassFK']
}

writers = {}
files = {}
for tbl, cols in tables_columns.items():
    path = os.path.join(OUTPUT_DIR, f"{tbl}.csv")
    f = open(path, 'w', newline='', encoding='utf-8')
    wr = csv.writer(f)
    wr.writerow(cols)
    files[tbl] = f
    writers[tbl] = wr

# --------------------------------------------------
# Conjuntos para garantizar unicidad/claves válidas
# --------------------------------------------------
port_set = set()
competition_set = set()
team_set = set()
car_plate_set = set()

# --------------------------------------------------
# 5) Generación de datos y escritura en CSV
# --------------------------------------------------

# 5.1 CruiseLine
cruise_lines = []
for _ in range(counts['CruiseLine']):
    name   = safe_str(faker.company())
    fd     = faker.date_between(date(1950,1,1), date.today()-timedelta(days=1))
    phone  = random.randint(10**9, 10**10-1)
    rating = round(random.random()*5, 2)
    writers['CruiseLine'].writerow([name, fd, phone, rating])
    cruise_lines.append((name, fd))

# 5.2 Person
persons = list(range(1, counts['Person']+1))
for pid in persons:
    nm    = safe_str(faker.name())
    birth = faker.date_of_birth(minimum_age=18, maximum_age=90)
    email = safe_str(faker.email())
    phone = random.randint(10**9, 10**10-1)
    nat   = safe_str(faker.country())
    if cruise_lines and random.random() < 0.3:
        cl = random.choice(cruise_lines)
        writers['Person'].writerow([nm, birth, email, phone, nat, cl[0], cl[1]])
    else:
        writers['Person'].writerow([nm, birth, email, phone, nat, '', ''])

# 5.3 City
cities = list(range(1, counts['City']+1))
for _ in cities:
    cname   = safe_str(faker.city())
    country = safe_str(faker.country())
    pop     = random.randint(0,10_000_000)
    area    = round(random.uniform(10,5000),2)
    lat     = round(random.uniform(-90,90),6)
    lon     = round(random.uniform(-180,180),6)
    writers['City'].writerow([cname, country, pop, area, lat, lon])

# 5.4 Vehicle (y reparto de IDs)
vehicles = list(range(1, counts['Vehicle']+1))
colors   = ['red','green','blue','yellow','black','white']
for vid in vehicles:
    mdl = safe_str(faker.word())
    wgt = round(random.uniform(500,3000),2)
    col = random.choice(colors)
    odo = round(random.uniform(0,200000),2)
    writers['Vehicle'].writerow([mdl, wgt, col, odo])

idx       = 0
cs_ids    = vehicles[idx: idx+counts['CruiseShip']]; idx+=counts['CruiseShip']
bike_ids  = vehicles[idx: idx+counts['Bike']]; idx+=counts['Bike']
car_ids   = vehicles[idx: idx+counts['Car']]; idx+=counts['Car']
bus_ids   = vehicles[idx: idx+counts['CityBus']]

# 5.5 CruiseShip
cruise_ships = []
classes = ['first','second','third','economy']
for vid in cs_ids:
    spd = round(random.uniform(10,30),2)
    lng = round(random.uniform(100,400),2)
    cap = random.randint(100,5000)
    cls = random.choice(classes)
    writers['CruiseShip'].writerow([spd, lng, cap, cls, vid])
    cruise_ships.append(vid)

# 5.6 Bike
bikes = []
bike_pk = 1
for vid in bike_ids:
    ele = random.choice([True,False])
    fol = random.choice([True,False])
    lig = random.choice([True,False])
    fh  = round(random.uniform(14,24),2)
    writers['Bike'].writerow([ele, fol, lig, fh, vid])
    bikes.append(bike_pk)
    bike_pk += 1

# 5.7 Car (asegurar plate único)
cars = []
for vid in car_ids:
    while True:
        plate = safe_str(faker.bothify('???-#####'))
        if plate not in car_plate_set:
            car_plate_set.add(plate)
            break
    fuel  = random.choice(['gas','diesel','electric','hybrid','hydrogen'])
    doorc = random.randint(2,5)
    trunk = random.randint(100,1000)
    hp    = random.randint(60,500)
    doors = random.randint(1,5)
    ac    = random.choice([True,False])
    writers['Car'].writerow([plate, fuel, doorc, trunk, hp, doors, ac, vid])
    cars.append(plate)

# 5.8 DrivingLicenseRequest
dl_requests = list(range(1, counts['DrivingLicenseRequest']+1))
for lid in dl_requests:
    rd  = faker.date_between(date.today()-timedelta(days=365), date.today())
    fee = round(random.uniform(20,200),2)
    p   = random.choice(persons)
    writers['DrivingLicenseRequest'].writerow([rd, fee, p])

# 5.9 & 5.10 DrivingLicense / RejectedDrivingLicense
dl_licenses = []
P_APPROVED = 0.6
P_REJECTED = 0.3
for lid in dl_requests:
    r = random.random()
    if r < P_APPROVED:
        ad  = faker.date_between(date.today()-timedelta(days=365), date.today())
        pts = random.randint(0,15)
        writers['DrivingLicense'].writerow([lid, ad, pts])
        dl_licenses.append(lid)
    elif r < P_APPROVED + P_REJECTED:
        rej    = faker.date_between(date.today()-timedelta(days=365), date.today())
        reap   = faker.date_between(rej, rej+timedelta(days=365))
        reason = safe_str(faker.word())
        writers['RejectedDrivingLicense'].writerow([lid, rej, reap, reason])

# 5.11 CarRegistration
for _ in range(counts['CarRegistration']):
    reg = faker.date_between(date.today()-timedelta(days=365), date.today())
    exp = faker.date_between(reg+timedelta(days=1), reg+timedelta(days=365))
    pl  = random.choice(cars)
    li  = random.choice(dl_licenses)
    writers['CarRegistration'].writerow([reg, exp, pl, li])

# 5.12 CarOwnership
for _ in range(counts['CarOwnership']):
    bd    = faker.date_time_between(start_date='-2y', end_date='now')
    price = round(random.uniform(10000,100000),2)
    we    = (bd + timedelta(days=random.randint(365,3650))).date()
    typ   = random.choice(['buy','rental','lease'])
    pl    = random.choice(cars)
    p     = random.choice(persons)
    writers['CarOwnership'].writerow([bd, price, we, typ, pl, p])

# 5.13 Rental
for _ in range(counts['Rental']):
    st   = faker.date_time_between(start_date='-1y', end_date='now')
    dur  = random.randint(0,48)
    hrp  = round(random.uniform(5,50),2)
    bk   = random.choice(bikes)
    p    = random.choice(persons)
    writers['Rental'].writerow([st, dur, hrp, bk, p])

# 5.14 ShipAssignment
for _ in range(counts['ShipAssignment']):
    sd   = faker.date_between(date.today()-timedelta(days=365*5), date.today())
    ed   = faker.date_between(sd, sd+timedelta(days=365))
    cl   = random.choice(cruise_lines)
    ship = random.choice(cruise_ships)
    writers['ShipAssignment'].writerow([sd, ed, cl[0], cl[1], ship])

# 5.15 Port (asegurar PK único Name+CityFK)
ports = []
for _ in range(counts['Port']):
    while True:
        pname = safe_str(faker.city())
        cfk   = random.choice(cities)
        if (pname, cfk) not in port_set:
            port_set.add((pname, cfk))
            break
    tc   = random.randint(0,10)
    ms   = random.randint(100,500)
    area = round(random.uniform(1,100),2)
    writers['Port'].writerow([pname, tc, ms, area, cfk])
    ports.append((pname, cfk))

# 5.16 Voyage
voyages = []
for _ in range(counts['Voyage']):
    dep_name, dep_city = random.choice(ports)
    arr_name, arr_city = random.choice([p for p in ports if p != (dep_name, dep_city)])
    dd   = faker.date_between(date.today()-timedelta(days=365), date.today())
    ad   = faker.date_between(dd, dd+timedelta(days=30))
    dist = round(random.uniform(10,10000),2)
    sp   = random.choice(cruise_ships)
    writers['Voyage'].writerow([dd, ad, dist, dep_name, dep_city, arr_name, arr_city, sp])
    voyages.append((dd, ad, dep_name, dep_city, arr_name, arr_city, sp))

# 5.17 CruiseBooking
for _ in range(counts['CruiseBooking']):
    bk_date = faker.date_between(date.today()-timedelta(days=365), date.today())
    cab     = random.randint(1,5000)
    price   = round(random.uniform(100,10000),2)
    pm      = random.choice(['card','paypal','bank','cash','mobile'])
    per     = random.choice(persons)
    voy     = random.choice(voyages)
    writers['CruiseBooking'].writerow([
        bk_date, cab, price, pm, per,
        voy[0], voy[1], voy[2], voy[3], voy[4], voy[5], voy[6]
    ])

# 5.18 Residence
for _ in range(counts['Residence']):
    sd  = faker.date_between(date.today()-timedelta(days=365*5), date.today())
    ed_s = faker.date_between(sd, sd+timedelta(days=365*2)) if random.random()<0.8 else ''
    addr = safe_str(faker.street_address())
    psn  = random.choice(persons)
    cfk  = random.choice(cities)
    writers['Residence'].writerow([sd, ed_s, addr, psn, cfk])

# 5.19 Pool
pools   = list(range(1, counts['Pool']+1))
statuss = ['open','closed','maintenance','renovation']
for pid in pools:
    nm   = safe_str(faker.company())
    addr = safe_str(faker.address())
    mind = random.randint(0,5)
    maxd = mind + random.randint(0,10)
    st   = random.choice(statuss)
    c_fk = random.choice(cities)
    writers['Pool'].writerow([pid, nm, addr, mind, maxd, st, c_fk])

# 5.20 CruisePool
cruise_pool_ids = random.sample(pools, counts['CruisePool'])
for pid in cruise_pool_ids:
    dn  = random.randint(0,10)
    mc  = random.randint(0,500)
    wt  = round(random.uniform(20,35),2)
    sc  = random.randint(0,5)
    sh  = random.choice(cruise_ships)
    writers['CruisePool'].writerow([pid, dn, mc, wt, sc, sh])

# 5.21 CityPool
city_pool_ids = random.sample(pools, counts['CityPool'])
for pid in city_pool_ids:
    mc = random.randint(0,5000)
    ab = round(random.uniform(1000,1e6),2)
    af = safe_str(faker.word())
    fw = random.choice([True,False])
    writers['CityPool'].writerow([pid, mc, ab, af, fw])

# 5.22 OlympicPool
olympic_pool_ids = random.sample(pools, counts['OlympicPool'])
for pid in olympic_pool_ids:
    spcap = random.randint(0,10000)
    lanes = random.randint(1,20)
    writers['OlympicPool'].writerow([pid, spcap, lanes])

# 5.23 Entry
entries = []
for _ in range(counts['Entry']):
    ts    = faker.date_time_between(start_date='-1y', end_date='now')
    price = round(random.uniform(5,100),2)
    pm    = random.choice(['card','paypal','bank','cash','mobile'])
    disc  = round(random.uniform(0, price),2)
    dur   = random.randint(1,300)
    per   = random.choice(persons)
    pl    = random.choice(city_pool_ids)
    writers['Entry'].writerow([ts, price, pm, disc, dur, per, pl])
    entries.append((ts, per, pl))

# 5.24 SwimmingCompetition (asegurar PK único Name+Start+End)
competitions = []
for _ in range(counts['SwimmingCompetition']):
    while True:
        nm = safe_str(faker.word())
        sd = faker.date_between(date.today()-timedelta(days=365), date.today())
        ed = faker.date_between(sd, sd+timedelta(days=30))
        key = (nm, sd, ed)
        if key not in competition_set:
            competition_set.add(key)
            break
    rt    = round(random.uniform(20,200),2)
    prize = random.randint(0,100_000)
    writers['SwimmingCompetition'].writerow([nm, sd, ed, rt, prize])
    competitions.append((nm, sd, ed))

# 5.25 Team (asegurar PK único Name+CoachFK)
teams = []
for _ in range(counts['Team']):
    while True:
        nm    = safe_str(faker.word())
        coach = random.choice(persons)
        if (nm, coach) not in team_set:
            team_set.add((nm, coach))
            break
    cd     = faker.date_between(date(1950,1,1), date.today()-timedelta(days=1))
    cloth  = random.choice(colors)
    sport  = random.choice(['waterpolo','swimming','diving'])
    budg   = random.randint(0,1_000_000)
    em     = safe_str(faker.email())
    home   = random.choice(olympic_pool_ids)
    writers['Team'].writerow([nm, cd, cloth, sport, budg, em, coach, home])
    teams.append((nm, coach))

# 5.26 Participation
all_pairs = [(c, t) for c in competitions for t in teams]
participations = []
for comp, team in random.sample(all_pairs, counts['Participation']):
    rd = faker.date_between(comp[1], comp[2])
    rk = random.randint(1,100)
    rt = round(random.uniform(20,200),2)
    writers['Participation'].writerow([rd, rk, rt, comp[0], comp[1], comp[2], team[0], team[1]])
    participations.append((comp[0], comp[1], comp[2], team[0], team[1]))

# 5.27 Sanction
sanctions = []
for sid in range(1, counts['Sanction']+1):
    amt  = round(random.uniform(0,1000),2)
    desc = safe_str(faker.word())
    idt  = faker.date_between(date.today()-timedelta(days=365), date.today())
    edt_s = faker.date_between(idt, idt+timedelta(days=365)) if random.random()<0.5 else ''
    st    = random.choice(['created','active','expired'])
    writers['Sanction'].writerow([sid, amt, desc, idt, edt_s, st])
    sanctions.append(sid)

# 5.28 DrivingSanction
dr_ids = random.sample(sanctions, counts['DrivingSanction'])
for sid in dr_ids:
    rs   = round(random.uniform(0,200),2)
    pd   = random.randint(0,15)
    sdur = random.randint(0,365)
    perm = random.choice([True,False])
    lic  = random.choice(dl_licenses)
    writers['DrivingSanction'].writerow([sid, rs, pd, sdur, perm, lic])

# 5.29 SportSanction
remaining = [s for s in sanctions if s not in dr_ids]
sp_ids    = random.sample(remaining, counts['SportSanction'])
for sid in sp_ids:
    part = random.choice(participations)
    susp = random.randint(0,10)
    ref  = safe_str(faker.name())
    writers['SportSanction'].writerow([sid, susp, ref, part[0], part[1], part[2], part[3], part[4]])

# 5.30 PoolSanction
remaining = [s for s in remaining if s not in sp_ids]
ps_ids    = random.sample(remaining, counts['PoolSanction'])
for sid in ps_ids:
    if random.random()<0.5:
        bs_s = faker.date_between(date.today()-timedelta(days=365), date.today())
        be_s = faker.date_between(bs_s, bs_s+timedelta(days=365))
    else:
        bs_s, be_s = '', ''
    comp_req = random.randint(0,1000)
    cs_req   = random.randint(0,500)
    ent      = random.choice(entries)
    writers['PoolSanction'].writerow([sid, bs_s, be_s, comp_req, cs_req, ent[0], ent[1], ent[2]])

# 5.31 Membership
for _ in range(counts['Membership']):
    jd    = faker.date_between(date.today()-timedelta(days=365*3), date.today())
    ld_s  = faker.date_between(jd, jd+timedelta(days=365*2)) if random.random()<0.8 else ''
    fee   = random.randint(0,1000)
    pfreq = random.choice(['monthly','anual','weekly','quarterly'])
    ar    = random.choice([True,False])
    psn   = random.choice(persons)
    tm, coach = random.choice(teams)
    writers['Membership'].writerow([jd, ld_s, fee, pfreq, ar, psn, tm, coach])

# 5.32 CityBus (ya asegurado en versión previa)
plates_set  = set()
citybus_plates = []
for vid in bus_ids:
    while True:
        candidate = safe_str(faker.bothify('BUS-#####'))
        if candidate not in plates_set:
            plates_set.add(candidate)
            citybus_plates.append(candidate)
            break
    rn    = random.randint(0,100)
    seats = random.randint(10,100)
    fw    = random.choice([True,False])
    dr    = random.choice(persons)
    cfk   = random.choice(cities)
    writers['CityBus'].writerow([candidate, rn, seats, fw, vid, dr, cfk])

# 5.33 BusTrip
bustrips = []
for _ in range(counts['BusTrip']):
    dt       = faker.date_between(date.today()-timedelta(days=365), date.today())
    sa       = safe_str(faker.street_address())
    ea       = safe_str(faker.street_address())
    duration = random.randint(5,120)
    pl       = random.choice(citybus_plates)
    writers['BusTrip'].writerow([dt, sa, ea, duration, pl])
    bustrips.append((dt, sa, ea, pl))

# 5.34 BusPass
buspasses = []
for pid in range(1, counts['BusPass']+1):
    idt = faker.date_between(date.today()-timedelta(days=365), date.today())
    edt = faker.date_between(idt+timedelta(days=1), idt+timedelta(days=365))
    mod = random.choice(['single','round_trip','daily','weekly','monthly','annual'])
    rt  = random.randint(0,100)
    psn = random.choice(persons)
    writers['BusPass'].writerow([pid, idt, edt, mod, rt, psn])
    buspasses.append(pid)

# 5.35 BusTicket
for _ in range(counts['BusTicket']):
    it   = faker.date_time_between(start_date='-1y', end_date='now')
    trip = random.choice(bustrips)
    if random.random()<0.5:
        psn = random.choice(persons)
        pfk = ''
    else:
        psn = ''
        pfk = random.choice(buspasses)
    writers['BusTicket'].writerow([it, trip[0], trip[1], trip[2], trip[3], psn, pfk])

# --------------------------------------------------
# 6) Cerrar todos los CSV
# --------------------------------------------------
for f in files.values():
    f.close()

# --------------------------------------------------
# 7) Generar script SQL con COPY
# --------------------------------------------------
with open(SQL_SCRIPT, 'w', encoding='utf-8') as f:
    for tbl, cols in tables_columns.items():
        csv_path = os.path.join(OUTPUT_DIR, f"{tbl}.csv")
        cols_list = ', '.join(cols)
        f.write(
            f"COPY {tbl} ({cols_list}) "
            f"FROM '{csv_path}' "
            f"WITH (FORMAT csv, HEADER true);\n"
        )

print(f"¡Listo! Se han generado {len(tables_columns)} CSVs en {OUTPUT_DIR} y el script {SQL_SCRIPT}")

¡Listo! Se han generado 35 CSVs en /content/data y el script /content/load_data.sql


**Data loading**

After creating the CSV files and the `load_data.sql` script, we run it with a shell command on the database.

In [9]:
!sudo -u postgres psql -q -d exampledatabase -f load_data.sql

We can also see how much space the `/content/data/` folder consumes, as it contains the CSV files with the data of the inserted tuples.

In [10]:
!du -sh /content/data/

5.0M	/content/data/


Once we have inserted the data, we can perform queries on the database. Specifically, in the following cells, **example queries** will be defined, and their code can be found in the respective `sql` variable, which is then provided as input to the shell command that runs the query on the `ExampleDataBase` database.

In [11]:
# @title **Sample query**
# @markdown You can execute a custom query in this cell
sql = """
SELECT PersonFK, COUNT(*) AS NumRentals
FROM Rental
GROUP BY PersonFK
HAVING COUNT(*)>2;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 personfk | numrentals 
----------+------------
     6192 |          3
     2388 |          3
     6677 |          3
     2676 |          3
     5164 |          3
     8943 |          3
     3048 |          3
     4198 |          3
     4190 |          3
      405 |          3
     7084 |          3
     1404 |          3
     7850 |          3
(13 rows)



In [12]:
# @title **Query 1**
# @markdown Count the number of tuples within each table of the database schema
sql = """
SELECT 'cruiseline' AS table_name, COUNT(*) AS row_count
FROM cruiseline
UNION ALL
SELECT 'person', COUNT(*)
FROM person
UNION ALL
SELECT 'vehicle', COUNT(*)
FROM vehicle
UNION ALL
SELECT 'cruiseship', COUNT(*)
FROM cruiseship
UNION ALL
SELECT 'bike', COUNT(*)
FROM bike
UNION ALL
SELECT 'car', COUNT(*)
FROM car
UNION ALL
SELECT 'drivinglicenserequest', COUNT(*)
FROM drivinglicenserequest
UNION ALL
SELECT 'drivinglicense', COUNT(*)
FROM drivinglicense
UNION ALL
SELECT 'rejecteddrivinglicense', COUNT(*)
FROM rejecteddrivinglicense
UNION ALL
SELECT 'carregistration', COUNT(*)
FROM carregistration
UNION ALL
SELECT 'carownership', COUNT(*)
FROM carownership
UNION ALL
SELECT 'rental', COUNT(*)
FROM rental
UNION ALL
SELECT 'shipassignment', COUNT(*)
FROM shipassignment
UNION ALL
SELECT 'city', COUNT(*)
FROM city
UNION ALL
SELECT 'port', COUNT(*)
FROM port
UNION ALL
SELECT 'voyage', COUNT(*)
FROM voyage
UNION ALL
SELECT 'cruisebooking', COUNT(*)
FROM cruisebooking
UNION ALL
SELECT 'residence', COUNT(*)
FROM residence
UNION ALL
SELECT 'pool', COUNT(*)
FROM pool
UNION ALL
SELECT 'cruisepool', COUNT(*)
FROM cruisepool
UNION ALL
SELECT 'citypool', COUNT(*)
FROM citypool
UNION ALL
SELECT 'olympicpool', COUNT(*)
FROM olympicpool
UNION ALL
SELECT 'entry', COUNT(*)
FROM entry
UNION ALL
SELECT 'swimmingcompetition', COUNT(*)
FROM swimmingcompetition
UNION ALL
SELECT 'team', COUNT(*)
FROM team
UNION ALL
SELECT 'participation', COUNT(*)
FROM participation
UNION ALL
SELECT 'sanction', COUNT(*)
FROM sanction
UNION ALL
SELECT 'drivingsanction', COUNT(*)
FROM drivingsanction
UNION ALL
SELECT 'sportsanction', COUNT(*)
FROM sportsanction
UNION ALL
SELECT 'poolsanction', COUNT(*)
FROM poolsanction
UNION ALL
SELECT 'membership', COUNT(*)
FROM membership
UNION ALL
SELECT 'citybus', COUNT(*)
FROM citybus
UNION ALL
SELECT 'bustrip', COUNT(*)
FROM bustrip
UNION ALL
SELECT 'buspass', COUNT(*)
FROM buspass
UNION ALL
SELECT 'busticket', COUNT(*)
FROM busticket;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

       table_name       | row_count 
------------------------+-----------
 cruiseline             |      1000
 person                 |     10000
 vehicle                |      7000
 cruiseship             |      1000
 bike                   |      2000
 car                    |      2000
 drivinglicenserequest  |      4000
 drivinglicense         |      2440
 rejecteddrivinglicense |      1180
 carregistration        |      2000
 carownership           |      2000
 rental                 |      2000
 shipassignment         |      2000
 city                   |      2000
 port                   |      2000
 voyage                 |      4000
 cruisebooking          |      4000
 residence              |      4000
 pool                   |      2000
 cruisepool             |      1000
 citypool               |      1000
 olympicpool            |      1000
 entry                  |      4000
 swimmingcompetition    |      1000
 team                   |      2000
 participation          | 

In [13]:
# @title **Query 2**
# @markdown Persons who have never carried out any bike rental.

# @markdown **Output:** all attributes from `Person` (one row per person).

sql = """
SELECT *
FROM Person P
WHERE P.PersonID NOT IN (SELECT PersonFK FROM Rental);
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

Streaming output truncated to the last 5000 lines.
     3893 | Melissa Hernandez           | 1936-08-01 | wbrown@example.com               | 5151472139 | Bolivia                          |                                  | 
     3894 | Robert Jenkins              | 1960-01-31 | john44@example.org               | 6476018294 | French Polynesia                 | Church, Burns and Davis          | 1959-07-01
     3895 | Samuel Maldonado            | 1979-06-07 | avilamichael@example.com         | 4499343813 | Kazakhstan                       |                                  | 
     3896 | Alex Butler                 | 1938-11-18 | ptyler@example.com               | 4045448624 | Cape Verde                       |                                  | 
     3897 | Julia Webb                  | 1997-06-17 | mallory90@example.com            | 9266337131 | Marshall Islands                 |                                  | 
     3898 | Kelly Smith                 | 1943-11-11 | jbowen@example

In [14]:
# @title **Query 3**
# @markdown Persons ordered from highest to lowest by the total number of driving license requests they have made.

# @markdown **Output:** `PersonID`, person’s name (`Name`), and number of requests (`NumRequests`).

sql = """
SELECT P.PersonID,
    (
        SELECT Name
        FROM Person
        WHERE PersonID = P.PersonID
    ),
    COUNT(*) AS NumRequests
FROM Person P INNER JOIN DrivingLicenseRequest DLR ON P.PersonID = DLR.PersonFK
GROUP BY P.PersonID
ORDER BY NumRequests DESC;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 personid |            name             | numrequests 
----------+-----------------------------+-------------
     1103 | Kathy Evans                 |           5
     6206 | Carl Thomas                 |           5
     4777 | Russell Yu                  |           4
     4889 | Christian Torres            |           4
     2936 | Shane Gonzalez              |           4
     1113 | Matthew Golden              |           4
     3051 | Amanda Palmer               |           3
     6722 | Samantha Hernandez          |           3
     5956 | Anthony Mitchell            |           3
     8158 | Jeremy Adams                |           3
     9505 | Gary Thomas                 |           3
     1596 | James Jones                 |           3
     5695 | Jose Johnson                |           3
     2306 | Molly Williamson            |           3
     2426 | Christopher Ward            |           3
     5567 | Timothy Strong              |           3
     6144 | Jennifer Marks

In [15]:
# @title **Query 3, alternative solution**

sql = """
WITH RequestCount AS (
    SELECT P.PersonID, COUNT(*) AS NumRequests
    FROM Person P INNER JOIN DrivingLicenseRequest DLR ON P.PersonID = DLR.PersonFK
    GROUP BY P.PersonID
)
SELECT P.PersonID, P.Name, RC.NumRequests
FROM RequestCount RC INNER JOIN Person P ON RC.PersonID = P.PersonID
ORDER BY NumRequests DESC;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 personid |            name             | numrequests 
----------+-----------------------------+-------------
     6206 | Carl Thomas                 |           5
     1103 | Kathy Evans                 |           5
     4777 | Russell Yu                  |           4
     2936 | Shane Gonzalez              |           4
     1113 | Matthew Golden              |           4
     4889 | Christian Torres            |           4
     2662 | Beverly Lewis               |           3
     2330 | Rebecca Romero              |           3
     4732 | Marco Clark II              |           3
     1691 | Olivia Goodwin              |           3
     5801 | Cheryl Flynn                |           3
     1540 | Nicholas Johnson            |           3
     8735 | Jerry George                |           3
     8947 | Kelly Zimmerman             |           3
     6524 | Mrs. Kimberly Martin        |           3
     3168 | Melissa Adams               |           3
     3052 | Tonya Figueroa

In [16]:
# @title **Query 4**
# @markdown Persons who have resided in the same city on more than one occasion (with different start dates).
# @markdown **Output:** `PersonFK`, `CityFK`.

sql = """
SELECT R1.PersonFK, R1.CityFK
FROM Residence R1 INNER JOIN Residence R2 ON R1.PersonFK = R2.PersonFK AND R1.CityFK = R2.CityFK
WHERE R1.StartDate<R2.StartDate;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 personfk | cityfk 
----------+--------
     9803 |   1934
(1 row)



In [17]:
# @title **Query 4, alternative solution**

sql = """
SELECT DISTINCT R1.PersonFK, R1.CityFK
FROM Residence R1 INNER JOIN Residence R2 ON R1.PersonFK = R2.PersonFK
    AND R1.CityFK = R2.CityFK
    AND R1.StartDate <> R2.StartDate;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 personfk | cityfk 
----------+--------
     9803 |   1934
(1 row)



In [18]:
# @title **Query 5**
# @markdown Bikes that are currently not rented. As a hint, you may find it useful to work with the `epoch` values of a timestamp (seconds since 1970-01-01 00:00:00 UTC) when handling time calculations.

# @markdown **Output:** all attributes from `Bike`.

sql = """
SELECT B.*
FROM Bike B
WHERE NOT EXISTS (
  SELECT *
  FROM Rental R
  WHERE R.BikeFK = B.BikeID
    AND EXTRACT(EPOCH FROM CURRENT_TIMESTAMP) >= EXTRACT(EPOCH FROM R.StartTimestamp)
    AND EXTRACT(EPOCH FROM CURRENT_TIMESTAMP) <  EXTRACT(EPOCH FROM R.StartTimestamp) + R.Duration * 3600
);
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 bikeid | electric | foldable | haslights | frameheight | vehicleid 
--------+----------+----------+-----------+-------------+-----------
    251 | f        | t        | t         |       19.63 |      1251
    264 | t        | t        | f         |       18.74 |      1264
   1350 | t        | f        | t         |       14.26 |      2350
   1070 | f        | t        | t         |       20.22 |      2070
   1209 | t        | t        | t         |        14.5 |      2209
    496 | t        | t        | f         |       16.79 |      1496
   1101 | f        | t        | t         |       16.97 |      2101
    276 | f        | f        | f         |       20.47 |      1276
    599 | f        | t        | f         |       14.71 |      1599
   1083 | t        | f        | t         |       15.66 |      2083
   1789 | t        | f        | f         |       16.47 |      2789
   1820 | t        | f        | f         |       20.72 |      2820
    437 | f        | f        | f         |   

In [19]:
# @title **Query 6**
# @markdown Number of rentals made, distinguishing between electric and non-electric bikes.

# @markdown **Output:** whether the bike is electric (`Electric`), and number of rentals of the corresponding type of bike (`NumRentals`).


sql = """
SELECT B.Electric, COUNT(*) AS NumRentals
FROM Rental R INNER JOIN Bike B ON R.BikeFK = B.BikeID
GROUP BY B.Electric;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 electric | numrentals 
----------+------------
 f        |        971
 t        |       1029
(2 rows)



In [20]:
# @title **Query 7**
# @markdown Cities where all their pools are open.

# @markdown **Output:** `CityID`, city name (`Name`).

sql = """
SELECT C.CityID, C.Name
FROM City C
WHERE NOT EXISTS (
    SELECT *
    FROM Pool P
    WHERE P.CityFK = C.CityID
      AND P.Status <> 'open'
);
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 cityid |           name           
--------+--------------------------
   1548 | Lake Matthewshire
   1350 | New Reginamouth
   1209 | Cooperland
   1080 | West Ariel
    455 | North Wendy
   1420 | East Davidfort
   1087 | New Henrybury
   1532 | South Logan
   1727 | Wagnertown
   1288 | Sarahburgh
   1083 | Riveramouth
   1789 | Harpertown
   1124 | South Michael
    151 | Jasmineton
    921 | Zacharymouth
    253 | Alexanderberg
   1269 | South Donna
   1283 | South Adriennefort
    101 | Lake Annside
    847 | Caitlinborough
   1314 | Brianberg
     25 | South Williamside
   1048 | Lake Thomasshire
   1253 | Terranceborough
    359 | Jasonstad
   1879 | Hernandezmouth
   1732 | Lake Michael
   1640 | Lake Michaelmouth
    196 | New Sherryfurt
   1741 | Graceshire
   1403 | North Dylanmouth
   1945 | North Hannahberg
    969 | Raymondview
   1587 | Jordanview
    414 | South John
   1116 | South Brianton
   1920 | South Robin
    449 | Zacharybury
    667 | Jamiemouth
   1487 | Jo

In [21]:
# @title **Query 8**
# @markdown Teams whose budget is above the average budget of the teams in the same sport.

# @markdown **Output:** team name (`Name`), sport (`Sport`), and budget (`Budget`).

sql = """
SELECT T.Name, T.Sport, T.Budget
FROM Team T
WHERE T.Budget > (
    SELECT AVG(T2.Budget)
    FROM Team T2
    WHERE T2.Sport = T.Sport
)
ORDER BY Budget;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

      name      |   sport   | budget 
----------------+-----------+--------
 once           | swimming  | 489657
 conference     | swimming  | 492515
 level          | swimming  | 493003
 their          | waterpolo | 500507
 majority       | swimming  | 500650
 somebody       | swimming  | 501121
 green          | waterpolo | 501286
 yet            | swimming  | 501837
 right          | swimming  | 502731
 arm            | waterpolo | 502868
 ten            | swimming  | 502924
 million        | waterpolo | 503344
 trial          | swimming  | 503744
 beyond         | waterpolo | 503798
 hope           | swimming  | 504402
 future         | waterpolo | 504450
 size           | swimming  | 505691
 design         | swimming  | 506961
 audience       | waterpolo | 507445
 lot            | swimming  | 508823
 significant    | swimming  | 509008
 bad            | waterpolo | 509395
 food           | waterpolo | 510024
 technology     | waterpolo | 511068
 truth          | swimming  | 511163

In [22]:
# @title **Query 9**
# @markdown Persons holding a driving license who have never received a driving sanction.

# @markdown **Output:** `PersonID`, person’s name (`Name`).

sql = """
SELECT P.PersonID, P.Name
FROM Person P INNER JOIN DrivingLicense DL ON P.PersonID = DL.LicenseID
WHERE DL.LicenseID NOT IN (
    SELECT LicenseFK FROM DrivingSanction
);
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 personid |           name           
----------+--------------------------
        5 | Courtney Lewis
       13 | David Reynolds
       24 | Edward Smith
       31 | Lisa Brown
       48 | Stephen Anderson
       49 | Joseph Patterson
       50 | Kimberly Cain
       51 | Pamela Adams
       54 | Brittany Robinson
       55 | Linda Martin
       62 | Jennifer Miller
       66 | Elizabeth Bridges
       75 | Brooke Conner
       76 | Jeanne Davis
       78 | Natasha Jones
       79 | Angela Baird
       80 | Jose Steele
       87 | Jose Wong
       96 | Michael Schaefer
       99 | Sarah Cox
      113 | Timothy Thomas
      115 | Troy Gross
      116 | Richard Clark
      119 | Linda Irwin
      127 | Debra Berger
      141 | Isabel Coleman
      142 | Connie Saunders
      144 | Jennifer Rose
      145 | Donald Hodge
      146 | Debbie Chen
      153 | Sydney Morris
      154 | Timothy Lane
      155 | Diane Leach
      158 | Mark Franklin
      159 | Carlos Edwards
      168 | Madiso

In [23]:
# @title **Query 10**
# @markdown Total revenue obtained for each payment method from cruise bookings on voyages with a distance of at least 1000 km.

# @markdown **Output:** payment method (`PaymentMethod`), total revenue (`TotalRevenue`).

sql = """
SELECT CB.PaymentMethod, SUM(CB.Price) AS TotalRevenue
FROM CruiseBooking CB INNER JOIN Voyage V
ON (CB.DepartureDateFK, CB.ArrivalDateFK,
      CB.DepartureNameFK, CB.DepartureCityFK,
      CB.ArrivalNameFK,   CB.ArrivalCityFK,
      CB.ShipFK)
   = (V.DepartureDate, V.ArrivalDate,
      V.DepartureNameFK, V.DepartureCityFK,
      V.ArrivalNameFK,   V.ArrivalCityFK,
      V.ShipFK)
WHERE V.Distance >= 1000
GROUP BY CB.PaymentMethod;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 paymentmethod |    totalrevenue    
---------------+--------------------
 mobile        |  3879371.319999999
 paypal        |  3515334.679999997
 cash          |  3567471.449999996
 bank          | 3490638.3800000027
 card          | 3659198.4199999995
(5 rows)



In [24]:
# @title **Query 11**
# @markdown Swimming competitions in which at least one participating team has matched or surpassed the competition’s record time.

# @markdown **Output:** competition name (`Name`), start date (`StartDate`), and end date (`EndDate`).

sql = """
SELECT DISTINCT SC.Name, SC.StartDate, SC.EndDate
FROM SwimmingCompetition SC INNER JOIN Participation P
  ON (P.NameFK, P.StartDateFK, P.EndDateFK) = (SC.Name, SC.StartDate, SC.EndDate)
WHERE P.RecordedTime >= SC.RecordTime;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

      name      | startdate  |  enddate   
----------------+------------+------------
 hair           | 2024-11-20 | 2024-11-22
 writer         | 2025-04-19 | 2025-05-05
 mouth          | 2025-05-25 | 2025-06-06
 character      | 2024-12-22 | 2024-12-28
 environment    | 2024-09-13 | 2024-09-24
 music          | 2024-08-27 | 2024-09-25
 increase       | 2025-02-12 | 2025-03-04
 growth         | 2025-03-04 | 2025-03-20
 oil            | 2024-11-10 | 2024-11-24
 security       | 2025-02-18 | 2025-03-19
 government     | 2024-10-17 | 2024-10-21
 worker         | 2025-03-16 | 2025-04-09
 really         | 2024-10-05 | 2024-10-05
 carry          | 2025-06-16 | 2025-07-06
 second         | 2024-12-20 | 2024-12-21
 magazine       | 2025-01-29 | 2025-02-01
 drop           | 2025-05-16 | 2025-06-09
 page           | 2025-06-04 | 2025-06-24
 positive       | 2025-08-04 | 2025-08-13
 similar        | 2025-03-04 | 2025-03-10
 which          | 2024-11-21 | 2024-12-02
 democratic     | 2025-07-22 | 2

In [25]:
# @title **Query 12**
# @markdown For each city, percentage of its pools that are currently open.

# @markdown **Output:** `CityID`, city name (`Name`), and percentage of open pools (`Ratio`).

sql = """
SELECT C.CityID, C.Name,
    100.0 * OpenPools.OpenCount / PoolCount.TotalCount AS Ratio
FROM City C INNER JOIN (
  SELECT CityFK, COUNT(*) AS TotalCount
  FROM Pool
  GROUP BY CityFK
) PoolCount ON PoolCount.CityFK = C.CityID
INNER JOIN (
  SELECT CityFK, COUNT(*) AS OpenCount
  FROM Pool
  WHERE Status = 'open'
  GROUP BY CityFK
) OpenPools ON OpenPools.CityFK = C.CityID;
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 cityid |         name          |        ratio         
--------+-----------------------+----------------------
   1798 | Port Davidview        |  50.0000000000000000
   1560 | Port Whitneytown      |  50.0000000000000000
    951 | Michaelbury           |  50.0000000000000000
   1284 | North Ashley          |  50.0000000000000000
    770 | New Joseph            |  33.3333333333333333
   1589 | North Melissa         |  50.0000000000000000
   1493 | New Gerald            | 100.0000000000000000
   1811 | West Javier           |  50.0000000000000000
   1175 | Port James            | 100.0000000000000000
    775 | Robertmouth           | 100.0000000000000000
    791 | Hayesfort             |  66.6666666666666667
   1830 | Jesusstad             | 100.0000000000000000
    940 | Rachaelland           |  50.0000000000000000
   1963 | East Christopher      | 100.0000000000000000
    161 | Valerieshire          |  66.6666666666666667
   1595 | New Erin              | 100.0000000000000000
    632 

In [26]:
# @title **Query 13**
# @markdown Persons whose name or email contains the string "gary" (you can use the `LOWER()` function to make it case-insensitive).

# @markdown **Output:** all attributes from `Person` (one row per person).

sql = """
SELECT *
FROM Person P
WHERE LOWER(P.Email) LIKE '%gary%' OR LOWER(P.Name) LIKE '%gary%';
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 personid |        name        |   birth    |            email            |   phone    |           nationality            |            namefk             | foundationdatefk 
----------+--------------------+------------+-----------------------------+------------+----------------------------------+-------------------------------+------------------
      314 | Gary Edwards       | 1960-05-04 | margaretknox@example.org    | 6251496763 | British Virgin Islands           | Gregory Inc                   | 1997-07-08
      580 | Gary Welch         | 1972-11-12 | barbara26@example.net       | 1589269854 | Singapore                        |                               | 
      630 | Gary Klein         | 1986-02-16 | kortega@example.net         | 1181054644 | United States Virgin Islands     |                               | 
      795 | Nicole Marsh       | 1985-05-07 | boltongary@example.org      | 2421868429 | Burkina Faso                     |                               | 
      867 | Ga